[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/crunchdao/quickstarters/blob/master/competitions/structural-break/quickstarters/baseline/baseline.ipynb)

![Banner](https://raw.githubusercontent.com/crunchdao/quickstarters/refs/heads/master/competitions/structural-break/assets/banner.webp)

# ADIA Lab Structural Break Challenge

## Challenge Overview

Welcome to the ADIA Lab Structural Break Challenge! In this challenge, you will analyze univariate time series data to determine whether a structural break has occurred at a specified boundary point.

### What is a Structural Break?

A structural break occurs when the process governing the data generation changes at a certain point in time. These changes can be subtle or dramatic, and detecting them accurately is crucial across various domains such as climatology, industrial monitoring, finance, and healthcare.

![Structural Break Example](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/competitions/structural-break/quickstarters/baseline/images/example.png)

### Your Task

For each time series in the test set, you need to predict a score between `0` and `1`:
- Values closer to `0` indicate no structural break at the specified boundary point;
- Values closer to `1` indicate a structural break did occur.

### Evaluation Metric

The evaluation metric is [ROC AUC (Area Under the Receiver Operating Characteristic Curve)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html), which measures the performance of detection algorithms regardless of their specific calibration.

- ROC AUC around `0.5`: No better than random chance;
- ROC AUC approaching `1.0`: Perfect detection.

# Setup

The first steps to get started are:
1. Get the setup command
2. Execute it in the cell below

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Reveal token](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/reveal-token.gif)

In [3]:
# Go to: https://hub.crunchdao.com/competitions/structural-break/submit/notebook?projectName=stormy-ape
# Get new token to run model
%pip install crunch-cli --upgrade --quiet --progress-bar off
!crunch setup-notebook structural-break kAmtg1KZQg7VNdCE6ovpE5Yc

delete /content/.crunchdao
main.py: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/submissions/21347/main.py (2945 bytes)
notebook.ipynb: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/submissions/21347/notebook.ipynb (85491 bytes)
requirements.txt: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/submissions/21347/requirements.original.txt (152 bytes)
data/X_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/y_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_test.reduced.parquet: download from https:crunchdao--compet

# Your model

## Setup

In [4]:
# Import your dependencies
import os
import typing

import joblib
import pandas as pd
import scipy
import sklearn.metrics

import crunch

import numpy as np
from typing import Tuple, Literal, List, Dict, Any, Optional, Union
from pydantic import BaseModel, Field, field_validator
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.base import BaseEstimator

import xgboost as xgb
import lightgbm as lgb
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.functional import binary_cross_entropy_with_logits
from rich import print


In [ ]:

# class definitions
class SyntheticDataGenerator(BaseModel):
    n_series: int
    pct_true: float  # Between 0 and 1
    min_length: int = 50
    max_length: int = 200
    seed: int = 42

    class Config:
        arbitrary_types_allowed = True

    def _random_distribution(self, name: str, size: int, params: dict) -> np.ndarray:
        dist_map = {
            'normal': lambda: np.random.normal(params.get('loc', 0), params.get('scale', 1), size),
            't': lambda: stats.t.rvs(df=params.get('df', 5), size=size),
            'exponential': lambda: np.random.exponential(params.get('scale', 1), size),
            'binomial': lambda: np.random.binomial(n=params.get('n', 10), p=params.get('p', 0.5), size=size),
        }
        if name not in dist_map:
            raise ValueError(f"Unsupported distribution: {name}")
        return dist_map[name]()

    def _generate_series(self, id_val: int, has_break: bool) -> Tuple[pd.DataFrame, bool]:
        np.random.seed(self.seed + id_val)

        length = np.random.randint(self.min_length, self.max_length + 1)
        breakpoint = np.random.randint(length // 3, length - 10) if has_break else np.random.randint(length // 2, length)

        # Randomly pick a base distribution
        dist_name = np.random.choice(['normal', 't', 'exponential', 'binomial'])
        base_params = {
            'normal': {'loc': 0, 'scale': 1},
            't': {'df': 5},
            'exponential': {'scale': 1},
            'binomial': {'n': 10, 'p': 0.5}
        }[dist_name]

        # Generate values
        pre_values = self._random_distribution(dist_name, breakpoint, base_params)

        if has_break:
            # Change distribution parameters
            changed_params = {
                'normal': {'loc': 1, 'scale': 1.5},
                't': {'df': 2},
                'exponential': {'scale': 2},
                'binomial': {'n': 10, 'p': 0.8}
            }[dist_name]
            post_values = self._random_distribution(dist_name, length - breakpoint, changed_params)
        else:
            post_values = self._random_distribution(dist_name, length - breakpoint, base_params)

        values = np.concatenate([pre_values, post_values])
        period = np.array([0]*breakpoint + [1]*(length - breakpoint))

        df = pd.DataFrame({
            'value': values,
            'period': period
        }, index=pd.MultiIndex.from_product([[id_val], range(length)], names=['id', 'time']))

        return df, has_break

    def generate(self) -> Tuple[pd.DataFrame, pd.Series]:
        true_count = int(self.pct_true * self.n_series)
        false_count = self.n_series - true_count
        labels = [True]*true_count + [False]*false_count
        np.random.shuffle(labels)

        series_list = []
        y_dict = {}

        for id_val, has_break in enumerate(labels):
            df, label = self._generate_series(id_val, has_break)
            series_list.append(df)
            y_dict[id_val] = label

        X = pd.concat(series_list)
        y = pd.Series(y_dict, name='structural_breakpoint')

        return X, y

class ETLPipeline(BaseModel):
    X_train: pd.DataFrame
    y_train: Optional[pd.Series] = pd.Series()

    class Config:
        arbitrary_types_allowed = True  # Allow non-pydantic types like pd.DataFrame

    @field_validator('X_train')
    def validate_X_train(cls, v):
        if not isinstance(v.index, pd.MultiIndex):
            raise ValueError("X_train must have a MultiIndex of ['id', 'time']")
        if 'value' not in v.columns or 'period' not in v.columns:
            raise ValueError("X_train must contain 'value' and 'period' columns")
        return v

    @field_validator('y_train')
    def validate_y_train(cls, v):
        if not isinstance(v, pd.Series):
            raise ValueError("y_train must be a pandas Series")
        if v.dtype != 'bool':
            raise ValueError("y_train must be of dtype 'bool'")
        return v

    def get_ids(self) -> list:
        """Returns a list of all unique ids in the training set."""
        return list(self.X_train.index.get_level_values('id').unique())

    def get_series_by_id(self, id_val: int) -> pd.DataFrame:
        """Returns the time series data for a specific id."""
        if (id_val not in self.y_train.index) & (len(self.y_train)>0):
            raise ValueError(f"id {id_val} not found in y_train")
        try:
            return self.X_train.loc[id_val]
        except KeyError:
            raise ValueError(f"id {id_val} not found in X_train")

    def get_target_by_id(self, id_val: int) -> bool:
        """Returns the target value for a specific id."""
        return self.y_train.loc[id_val]

    def get_structural_breakdown(self) -> pd.Series:
        """Returns the proportion of True/False in y_train"""
        return self.y_train.value_counts(normalize=True).rename("proportion")

class FeatureGenerator(BaseModel):
    etl: ETLPipeline

    class Config:
        arbitrary_types_allowed = True

    def extract_features_for_id_testing(self, id_val: int) -> Dict[str, Any]:
        """Extracts features for a single time series id"""

        ts = self.etl.get_series_by_id(id_val)
        # print(f"id {id_val}, {id_change}")

        # Find index where period first changes from 0 to 1
        change_indices = ts.index[ts['period'].diff() == 1].tolist()

        if (not change_indices):
            # No regime change occurred — use whole series
            before = ts
            after = ts # None
            change_point = np.nan
        else:
            change_point = change_indices[0]
            before = ts.loc[:change_point]
            after = ts.loc[change_point + 1:] if change_point + 1 in ts.index else None

        features = {
            "id": id_val,
            "change_point_idx": change_point if not np.isnan(change_point) else -1,
            "mean_before_change": before['value'].mean(),
            "std_before_change": before['value'].std(),
            "length_before_change": len(before),
            "mean_after_change": after['value'].mean() if after is not None else np.nan,
            "std_after_change": after['value'].std() if after is not None else np.nan,
            "length_after_change": len(after) if after is not None else 0,
            # "skew_before_change": before['value'].skew(),
            # "skew_after_change": after['value'].skew(),
            # "kurtosis_before_change": before['value'].kurtosis(),
            # "kurtosis_after_change": after['value'].kurtosis()
        }

        features["delta_mean"] = (
            features["mean_after_change"] - features["mean_before_change"]
            if after is not None else np.nan
        )

        return features

    def extract_features_for_id_training(self, id_val: int) -> Dict[str, Any]:
        """Extracts features for a single time series id"""

        id_change = self.etl.y_train[id_val]
        ts = self.etl.get_series_by_id(id_val)
        # print(f"id {id_val}, {id_change}")

        # Find index where period first changes from 0 to 1
        change_indices = ts.index[ts['period'].diff() == 1].tolist()

        if (not change_indices) | (not id_change):
            # No regime change occurred — use whole series
            before = ts
            after = ts # None
            change_point = np.nan
        else:
            change_point = change_indices[0]
            before = ts.loc[:change_point]
            after = ts.loc[change_point + 1:] if change_point + 1 in ts.index else None

        features = {
            "id": id_val,
            "change_point_idx": change_point if not np.isnan(change_point) else -1,
            "mean_before_change": before['value'].mean(),
            "std_before_change": before['value'].std(),
            "length_before_change": len(before),
            "mean_after_change": after['value'].mean() if after is not None else np.nan,
            "std_after_change": after['value'].std() if after is not None else np.nan,
            "length_after_change": len(after) if after is not None else 0,
            # "skew_before_change": before['value'].skew(),
            # "skew_after_change": after['value'].skew(),
            # "kurtosis_before_change": before['value'].kurtosis(),
            # "kurtosis_after_change": after['value'].kurtosis()
        }

        features["delta_mean"] = (
            features["mean_after_change"] - features["mean_before_change"]
            if after is not None else np.nan
        )

        return features

    def generate_feature_dataframe(self) -> pd.DataFrame:
        """Generates features for all ids"""
        all_ids = self.etl.get_ids()
        feature_dicts = []

        for id_val in all_ids:
            try:
              if len(self.etl.y_train)>0:
                features = self.extract_features_for_id_training(id_val)
                feature_dicts.append(features)
              else:
                features = self.extract_features_for_id_testing(id_val)
                feature_dicts.append(features)
            except Exception as e:
                print(f"Skipping id {id_val} due to error: {e}")

        return pd.DataFrame(feature_dicts).set_index("id")

class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(1)

class MLModelPipeline(BaseModel):
    X: pd.DataFrame
    y: pd.Series
    X_train: Optional[pd.DataFrame] = None
    y_train: Optional[pd.Series] = None
    X_test: Optional[pd.DataFrame] = None
    y_test: Optional[pd.Series] = None
    model_name: str = Field(default="random_forest")
    model: Optional[Any] = None
    test_size: float = 0.2
    random_state: int = 42
    device: str = Field(default="cuda" if torch.cuda.is_available() else "cpu")

    class Config:
        arbitrary_types_allowed = True

    @field_validator('X')
    def validate_X(cls, v):
        if not isinstance(v, pd.DataFrame):
            raise ValueError("X must be a pandas DataFrame")
        return v

    @field_validator('y')
    def validate_y(cls, v):
        if not isinstance(v, pd.Series):
            raise ValueError("y must be a pandas Series")
        if v.dtype != bool:
            raise ValueError("y must be a boolean Series")
        return v

    def _initialize_model(self, input_dim: int = None):
        if self.model_name == "logistic_regression":
            return LogisticRegression(max_iter=1000)
        elif self.model_name == "random_forest":
            return RandomForestClassifier(n_estimators=1000)
        elif self.model_name == "xgboost":
            return xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
        elif self.model_name == "lightgbm":
            return lgb.LGBMClassifier()
        elif self.model_name == "mlp":
            if input_dim is None:
                raise ValueError("input_dim required for MLP model")
            return MLP(input_dim).to(self.device)
        else:
            raise ValueError(f"Unsupported model: {self.model_name}")

    def fit(self):
        print(f"Running model: {self.model_name}")

        # Align feature index with label index
        X_aligned = self.X.loc[self.y.index.intersection(self.X.index)]
        y_aligned = self.y.loc[X_aligned.index]

        X_train, X_test, y_train, y_test = train_test_split(
            X_aligned, y_aligned, test_size=self.test_size, random_state=self.random_state
        )

        self.X_train, self.X_test = X_train, X_test
        self.y_train, self.y_test = y_train, y_test

        if self.model_name == "mlp":
            input_dim = X_train.shape[1]
            self.model = self._initialize_model(input_dim)
            self._train_mlp(X_train, y_train)
        else:
            self.model = self._initialize_model()
            self.model.fit(X_train, y_train)

    def _train_mlp(self, X_train: pd.DataFrame, y_train: pd.Series):
        X_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y_train.values.astype(np.float32)).to(self.device)

        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        self.model.train()

        for epoch in range(30):  # small epochs for demo
            for xb, yb in dataloader:
                preds = self.model(xb)
                loss = binary_cross_entropy_with_logits(preds, yb)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

    def predict(self, X: Optional[pd.DataFrame] = None) -> np.ndarray:
        if X is None:
          X = self.X_test

        if self.model_name == "mlp":
            self.model.eval()
            with torch.no_grad():
                X_tensor = torch.tensor(X.values, dtype=torch.float32).to(self.device)
                logits = self.model(X_tensor)
                probs = torch.sigmoid(logits)
                return (probs > 0.5).cpu().numpy()
        else:
            return self.model.predict(X)

    def evaluate(self) -> Dict[str, Any]:
        preds = self.predict()
        if self.model_name == "mlp":
            with torch.no_grad():
                X_tensor = torch.tensor(self.X_test.values, dtype=torch.float32).to(self.device)
                probs = torch.sigmoid(self.model(X_tensor)).cpu().numpy()
        elif hasattr(self.model, "predict_proba"):
            probs = self.model.predict_proba(self.X_test)[:, 1]
        else:
            probs = preds

        report = classification_report(self.y_test, preds, output_dict=True)
        auc = roc_auc_score(self.y_test, probs) if probs is not None else None

        return {
            "classification_report": report,
            "roc_auc": auc,
            "model": self.model_name,
            "probs": probs
        }

In [5]:
# Load the Crunch Toolings
crunch = crunch.load_notebook()

loaded inline runner with module: <module '__main__'>

cli version: 6.4.4
available ram: 12.67 gb
available cpu: 2 core
----


## Understanding the Data

The dataset consists of univariate time series, each containing ~2,000-5,000 values with a designated boundary point. For each time series, you need to determine whether a structural break occurred at this boundary point.

The data was downloaded when you setup your local environment and is now available in the `data/` directory.

In [6]:
# Load the data simply
X_train, y_train, X_test = crunch.load_data()

data/X_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/X_test.reduced.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
data/y_test.reduced.parquet: already exists, file length match


### Understanding `X_train`

The training data is structured as a pandas DataFrame with a MultiIndex:

**Index Levels:**
- `id`: Identifies the unique time series
- `time`: The timestep within each time series

**Columns:**
- `value`: The actual time series value at each timestep
- `period`: A binary indicator where `0` represents the **period before** the boundary point, and `1` represents the **period after** the boundary point

In [7]:
X_train.tail(3)

value  period
id    time                  
10000 2136  0.000687       1
      2137  0.001640       1
      2138  0.001074       1

### Understanding `y_train`

This is a simple `pandas.Series` that tells if a dataset id has a structural breakpoint or not.

**Index:**
- `id`: the ID of the dataset

**Value:**
- `structural_breakpoint`: Boolean indicating whether a structural break occurred (`True`) or not (`False`)

In [8]:
y_train.tail(3)

,structural_breakpoint
id,
9998,False
9999,False
10000,True


### Understanding `X_test`

The test data is provided as a **`list` of `pandas.DataFrame`s** with the same format as [`X_train`](#understanding-X_test).

It is structured as a list to encourage processing records one by one, which will be mandatory in the `infer()` function.

In [12]:
print("Number of datasets:", len(X_test))
print(X_test[0].tail())

Number of datasets: 101

value  period
id    time                  
10001 2774 -0.013937       1
      2775 -0.015649       1
      2776 -0.009744       1
      2777  0.025375       1
      2778 -0.001515       1

## Strategy Implementation

There are multiple approaches you can take to detect structural breaks:

1. **Statistical Tests**: Compare distributions before and after the boundary point;
2. **Feature Engineering**: Extract features from both segments for comparison;
3. **Time Series Modeling**: Detect deviations from expected patterns;
4. **Machine Learning**: Train models to recognize break patterns from labeled examples.

The baseline implementation below uses a simple statistical approach: a t-test to compare the distributions before and after the boundary point.

### The `train()` Function

In this function, you build and train your model for making inferences on the test data. Your model must be stored in the `model_directory_path`.

The baseline implementation below doesn't require a pre-trained model, as it uses a statistical test that will be computed at inference time.

In [13]:
def train(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    model_directory_path: str,
):
    # For our baseline t-test approach, we don't need to train a model
    # This is essentially an unsupervised approach calculated at inference time
    model = None

    # You could enhance this by training an actual model, for example:
    # 1. Extract features from before/after segments of each time series
    # 2. Train a classifier using these features and y_train labels
    # 3. Save the trained model

    joblib.dump(model, os.path.join(model_directory_path, 'model.joblib'))

### The `infer()` Function

In the inference function, your trained model (if any) is loaded and used to make predictions on test data.

**Important workflow:**
1. Load your model;
2. Use the `yield` statement to signal readiness to the runner;
3. Process each dataset one by one within the for loop;
4. For each dataset, use `yield prediction` to return your prediction.

**Note:** The datasets can only be iterated once!

In [14]:
def infer(
    X_test: typing.Iterable[pd.DataFrame],
    model_directory_path: str,
):
    model = joblib.load(os.path.join(model_directory_path, 'model.joblib'))

    yield  # Mark as ready

    # X_test can only be iterated once.
    # Before getting the next dataset, you must predict the current one.
    for dataset in X_test:
        # Baseline approach: Compute t-test between values before and after boundary point
        # The negative p-value is used as our score - smaller p-values (larger negative numbers)
        # indicate more evidence against the null hypothesis that distributions are the same,
        # suggesting a structural break
        def t_test(u: pd.DataFrame):
            return -scipy.stats.ttest_ind(
                u["value"][u["period"] == 0],  # Values before boundary point
                u["value"][u["period"] == 1],  # Values after boundary point
            ).pvalue

        prediction = t_test(dataset)
        yield prediction  # Send the prediction for the current dataset

        # Note: This baseline approach uses a t-test to compare the distributions
        # before and after the boundary point. A smaller p-value (larger negative number)
        # suggests stronger evidence that the distributions are different,
        # indicating a potential structural break.

## Local testing

To make sure your `train()` and `infer()` function are working properly, you can call the `crunch.test()` function that will reproduce the cloud environment locally. <br />
Even if it is not perfect, it should give you a quick idea if your model is working properly.

In [88]:
crunch.test(
    # Uncomment to disable the train
    # force_first_train=False,

    # Uncomment to disable the determinism check
    # no_determinism_check=True,
)

ignoring cell #64: '(' was never closed (<unknown>, line 6)


18:57:27 no forbidden library found
18:57:27 
18:57:28 started
18:57:28 running local test
18:57:28 internet access isn't restricted, no check will be done
18:57:28 
18:57:30 starting unstructured loop...
18:57:30 executing - command=train


data/X_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/X_test.reduced.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
data/y_test.reduced.parquet: already exists, file length match


Running model: random_forest

Saving model in: resources

...

Model type: <class '__main__.MLModelPipeline'>

18:57:55 checking determinism by executing the inference again with 30% of the data (tolerance: 1e-08)
18:57:55 executing - command=infer


Reading model from: resources

Model type: <class '__main__.MLModelPipeline'>

18:57:57 determinism check: passed
18:57:57 save prediction - path=data/prediction.parquet
18:57:57 ended
18:57:57 duration - time=00:00:29
18:57:57 memory - before="1.41 GB" after="1.42 GB" consumed="11.8 MB"


In [87]:
def train(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    model_directory_path: str,
    model_name: Optional[str] = "random_forest"
):
    # For our baseline t-test approach, we don't need to train a model
    # This is essentially an unsupervised approach calculated at inference time
    etl = ETLPipeline(X_train=X_train, y_train=y_train)
    generator = FeatureGenerator(etl=etl)
    X_features = generator.generate_feature_dataframe()

    # Find rows without NaN values
    valid_index = X_features.loc[~X_features.isna().any(axis=1)].index
    X = X_features.loc[valid_index]
    y = etl.y_train[valid_index]

    model = MLModelPipeline(X=X, y=y, model_name=model_name)  # "logistic_regression", "random_forest", "xgboost", "lightgbm", "mlp"
    model.fit()
    # model = None

    # You could enhance this by training an actual model, for example:
    # 1. Extract features from before/after segments of each time series
    # 2. Train a classifier using these features and y_train labels
    # 3. Save the trained model

    print(f"Saving model in: {model_directory_path}")
    print(f"Model type: {type(model)}")
    joblib.dump(model, os.path.join(model_directory_path, 'model.joblib'))

def infer(
    X_test: typing.Iterable[pd.DataFrame],
    model_directory_path: str,
):
    print(f"Reading model from: {model_directory_path}")
    model = joblib.load(os.path.join(model_directory_path, 'model.joblib'))
    print(f"Model type: {type(model)}")

    yield  # Mark as ready

    # X_test can only be iterated once.
    # Before getting the next dataset, you must predict the current one.
    for dataset in X_test:
        etl = ETLPipeline(X_train=dataset)
        generator = FeatureGenerator(etl=etl)
        X_features = generator.generate_feature_dataframe()
        valid_index = X_features.loc[~X_features.isna().any(axis=1)].index
        X = X_features.loc[valid_index]
        prediction = model.predict(X)

        yield prediction  # Send the prediction for the current dataset

In [79]:
"""
# ETL pipeline
"""

etl = ETLPipeline(X_train=X_train, y_train=y_train)
# print(etl.get_ids()[:5])  # List first 5 ids
# print(etl.get_series_by_id(2).head())  # Time series for id 2
# print(etl.get_target_by_id(2))  # True or False
# print(etl.get_structural_breakdown())  # Proportion of structural breaks

"""
# Feature generator
"""

generator = FeatureGenerator(etl=etl)
X_features = generator.generate_feature_dataframe()

print(X_features.head(2))

# Assume X_features and etl.y_train are ready
valid_index = X_features.loc[~X_features.isna().any(axis=1)].index
X = X_features.loc[valid_index]
y = etl.y_train[valid_index]

model = MLModelPipeline(X=X, y=y, model_name="random_forest")  # "logistic_regression", "random_forest", "xgboost", "lightgbm", "mlp"
model.fit()

results = model.evaluate()
print(results["classification_report"])
print("AUC:", results["roc_auc"])


change_point_idx  mean_before_change  std_before_change  \
id                                                            
0                 -1            0.000013           0.006966   
1                 -1            0.000104           0.002475   

    length_before_change  mean_after_change  std_after_change  \
id                                                              
0                   1645           0.000013          0.006966   
1                   2529           0.000104          0.002475   

    length_after_change  delta_mean  
id                                   
0                  1645         0.0  
1                  2529         0.0

Running model: random_forest

> <ipython-input-72-4e104b82bc58>(334)predict()
    332     def predict(self, X: Optional[pd.DataFrame] = None) -> np.ndarray:
    333         import pdb; pdb.set_trace()
--> 334         if X is None:
    335           X = self.X_test
    336 

ipdb> c


{
    'False': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1399.0},
    'True': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 602.0},
    'accuracy': 1.0,
    'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 2001.0},
    'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 2001.0}
}

AUC: 1.0

In [84]:
type(model)
# model.X_test.dtypes

__main__.MLModelPipeline

In [80]:
etl = ETLPipeline(X_train=X_test[0])
generator = FeatureGenerator(etl=etl)
X_features = generator.generate_feature_dataframe()
valid_index = X_features.loc[~X_features.isna().any(axis=1)].index
X = X_features.loc[valid_index]
model.predict(X)

> <ipython-input-72-4e104b82bc58>(334)predict()
    332     def predict(self, X: Optional[pd.DataFrame] = None) -> np.ndarray:
    333         import pdb; pdb.set_trace()
--> 334         if X is None:
    335           X = self.X_test
    336 

ipdb> c


array([ True])

## Results

Once the local tester is done, you can preview the result stored in `data/prediction.parquet`.

In [19]:
prediction = pd.read_parquet("data/prediction.parquet")
prediction

,prediction
id,
10001,-0.590381
10002,-0.363831
10003,-0.731208
10004,-0.762609
10005,-0.527371
...,...
10097,-0.539917
10098,-0.843084
10099,-0.203762


### Local scoring

You can call the function that the system uses to estimate your score locally.

In [21]:
# Load the targets
target = pd.read_parquet("data/y_test.reduced.parquet")["structural_breakpoint"]

# Call the scoring function
sklearn.metrics.roc_auc_score(
    target,
    prediction,
)

np.float64(0.48450704225352115)

# Submit your Notebook

To submit your work, you must:
1. Download your Notebook from Colab
2. Upload it to the platform
3. Create a run to validate it

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Download and Submit Notebook](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/download-and-submit-notebook.gif)